# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil

# 8. Méthode split-apply-combine

La méthode **groupby()** permet d'agréger les données selon les valeurs identiques d'une ou plusieurs colonnes.

Elle renvoie un objet de type *DataFrameGroupBy* qui peut être interprété comme un dictionnaire d'objets de type *DataFrame* dont :
- les clefs sont les modalités des valeurs de la colonne (ou des colonnes) utilisée(s) pour éclater les données
- et les valeurs des sous-DataFrames  du DataFrame initial.

**Méthodologie** :
- **split** : partition des données en sous-groupes
- **apply** : application d'une fonction sur chaque groupe
- **combine** : agrégation des résultats

Voir la documentation : http://pandas.pydata.org/pandas-docs/stable/groupby.html

In [ ]:
# import des modules usuels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
pd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [ ]:
# chargement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                  index_col="Code INSEE")
geo = geo.sort_index()
geo.head()

In [ ]:
# on fabrique un objet de type DataFrameGroupBy
regions = geo.groupby("Région")
type(regions)

Un objet *DataFrameGroupBy* peut être vu comme un dictionnaire dont :
- les clefs sont les valeurs de la colonne utilisée pour éclater les données
- les valeurs sont des sous-DataFrame du *DataFrame* initial (sans la colonne ayant servie à éclater les données

In [ ]:
# accès au dictionnaire des groupes
type(regions.groups)

In [ ]:
# les clés sont les modalités de la colonne ayant servi à constituer les groupes
regions.groups.keys()

In [ ]:
# accès au nombre de groupes
regions.ngroups

La méthode **size()** permet de calculer l'effectif de chaque groupe.

In [ ]:
# effectifs des groupes
geo.groupby("Région").size()

On vérifie que la somme des effectifs vaut le nombre total de communes.

In [ ]:
# somme des effectifs
geo.groupby("Région").size().sum() == len(geo)

La méthode **describe()** fournit les informations statistiques sur les différents sous-groupes.

In [ ]:
# DataFrame
geo.describe()

In [ ]:
# describe par groupe
geo.groupby("Région").describe()

La méthode **get_group()** permet d'obtenir le sous-*DataFrame* correspondant au groupe.

In [ ]:
# accès à un groupe
geo.groupby("Région").get_group("CORSE").head()

<div class="alert alert-success">
<b>Exercice 1</b>
<ul>
    <li>Obtenir le groupe régional avec le plus petit effectif.</li>
</ul>
</div>

La méthode **aggregate()** ou bien **agg()** permet d'agréger toutes les valeurs par regroupement en passant une ou plusieurs fonctions de calcul qui peuvent être organisées dans une liste ou dans un dictionnaire.

In [ ]:
# somme de toutes les valeurs numériques agrégées
geo.groupby("Région").agg(np.mean)

In [ ]:
# moyenne et écart type de toutes les valeurs agrégées
geo.groupby("Région").agg([np.mean, np.std])

In [ ]:
# moyenne et écart type de toutes les valeurs agrégées
geo.groupby("Région").agg({'Altitude Moyenne': 'mean', 'Superficie': 'sum', 'Population': 'sum'})

La méthode **apply()** applique la fonction spécifiée à chacun des groupes d'un objet *DataFrameGroupBy* et combine l'ensemble en un nouveau *DataFrame*.

In [ ]:
# population de la plus grande ville par région
var = geo.groupby("Région").apply(lambda x: x['Population'].nlargest(1))
var

<div class="alert alert-success">
<b>Exercice 2</b>
<ul>
    <li>Etablir une fonction qui calcule le top 3 des villes les plus habitées d'une région.</li>
    <li>Utiliser la fonction pour limiter le <code>DataFrame</code> geo aux villes qui sont top 10 dans chaque région.</li>
    <li>Idem en sélectionnant un Statut particulier.</li>
</ul>

Il est possible de grouper selon plusieurs colonnes.

In [ ]:
# groupes multiples
geo.groupby(["Région", "Département"]).describe()

Il est possible d'utiliser la fonction `agg()` ou `aggregate()` sur une ou plusieurs colonnes en passant un dictionnaire de fonctions. On obtient alors un `DataFrame` avec les résultats ventilés par groupe et par colonne / clés de fonction.

In [ ]:
# synthèses différenciées sur plusieurs colonnes
regions.agg({"Superficie": np.sum, "Population": np.sum, "Altitude Moyenne": np.mean})

Il est également possible d'effectuer des regroupements selon des catégories numériques obtenues avec la fonction `digitize()` qui retourne pour chaque élément l'indice du segment (entre  et n).

In [ ]:
thd = pd.read_excel("FranceTHD_Open_Data_Observatoire_Juin2015.xlsx",
                    sheet_name="Communes",
                    header=1,
                    index_col="Code INSEE",
                    names=["Département", "Code INSEE", "Commune",
                                "1 Mbit", "3 Mbit", "8 Mbit", "30 Mbit", "100 Mbit",
                                "DSL 1 Mbit", "DSL 3 Mbit", "DSL 8 Mbit", "DSL 30 Mbit", "DSL 100 Mbit",
                                "Câble 1 Mbit", "Câble 3 Mbit", "Câble 8 Mbit", "Câble 30 Mbit", "Câble 100 Mbit",
                                "Fibre 1 Mbit", "Fibre 3 Mbit", "Fibre 8 Mbit", "Fibre 30 Mbit", "Fibre 100 Mbit"])
thd.head()

In [ ]:
# partition de [0,1] en 5 segments
bins = np.linspace(0, 1, 5)
bins

In [ ]:
# catégorisation de la colonne "1 Mbit" selon la partition donnée
cat = np.digitize(thd["1 Mbit"], bins)
cat

Par défaut, il s'agit d'une partition semi-ouverte à droite (*right=False*) :
- [0, 0.25[
- [0.25, 0.5[
- [0.5, 0.75[
- [0.75, 1.0[
- [1.0, 1.0]

In [ ]:
# regroupement selon la partition et calcul de la moyenne des valeurs des groupes
thd.groupby(cat).mean()

In [ ]:
# regroupement selon la partition et calcul de la moyenne et l'écart-type des valeurs des groupes
thd.groupby(cat).agg(['mean', 'std'])